In [ ]:
import tensorflow as tf
from tensorflow import keras
import os

# TensorFlow CNN - Lab Solutions

This Lab will use the "rock_paper_scissors" data. These are images of individuals displaying either a rock, paper or scissors hand gesture. There are 2520 training images (840 each), and 372 test images (124 each). Each is a 300 x 300 rgb image. There are also 33 additional images (in for_predictions folder) to do further validations. 

### Create a CNN

In [ ]:
model = tf.keras.models.Sequential([
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), padding='same', activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # 4 output neurons for multi-class output requires 'softmax' and will sum to 1. Each node will output its class' probability.
    tf.keras.layers.Dense(3, activation='softmax')
    ])


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Display a summary of the model

In [ ]:
model.summary()

### Normalize the data and generate labels for the images

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_data_generator = ImageDataGenerator(rescale=1/255)
validate_data_generator = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 200 using train_data_generator
train_generator = train_data_generator.flow_from_directory(
        'rock_paper_scissors/train',  # This is the source directory for training images 
                            # that contains subdirectories(whose names will be the labels) that contain images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=200,
        # Since we use 'sparse_categorical_crossentropy' loss, we need sparse classes
        class_mode='sparse')

# Flow validating images in batches of 25 using validate_data_generator
validate_generator = validate_data_generator.flow_from_directory(
        'rock_paper_scissors/test',  # This is the source directory for training images 
                            # that contains subdirectories(whose names will be the labels) that contain images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=100,
        # Since we use 'sparse_categorical_crossentropy' loss, we need sparse classes
        class_mode='sparse')

### Train the CNN (using the generated labels)
Fill in the steps for the training and validation generators. 

In [ ]:
# fit the model using the generated labels

history = model.fit_generator(
      train_generator,
      steps_per_epoch=13,     # the ceiling of: total_number_of_images/batch_size
                             # int(np.ceil(train_generator.n / float(batch_size)))
      epochs=5,
    
      validation_data = validate_generator,
      validation_steps=4,
      verbose=1)

### Manually validate the model 
Just fill in the tuple for target_size on line 13

In [ ]:
import numpy as np
from keras .preprocessing import image


class_labels = ['paper', 'rock', 'scissors']

# a list of images (from a directory/folder) that will be used to validate the model's predictions
validating_images = os.listdir("rock_paper_scissors/for_predictions")

# predicting images from a folder containing a set of unseen images
for file_name in validating_images:
    path = 'rock_paper_scissors/for_predictions/' + file_name
    img = image.load_img(path, target_size = (150,150))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    images = np.vstack([x])
    
    prediction = model.predict(images)
    
    out = prediction[0]
    print(file_name, ": ", class_labels[out.argmax()])
